In [1]:
import os
import re
import itertools
import csv
import shutil
import pandas as pd
from tabulate import tabulate
from functools import reduce

In [2]:
data_dir = '..\\data\\img_labeling_3rd_round_team\\'

In [3]:
def list_files(directory):
    label_files = sorted(os.listdir(directory))
    files = []
    label_files = [f for f in label_files if 'A00' in f]
    for f in label_files:
        files.append(f)
    
    return files

In [4]:
list_files(data_dir)
print("Total number of files extracted:", len(list_files(data_dir)))

Total number of files extracted: 5


In [5]:
col_names = ['imname', 'annotator']

In [6]:
#set1 = list_files(data_dir)
def image_set(directory, condition):
    set_n = list_files(data_dir)
    set_files = []
    set_n = [f for f in set_n if condition in f]
    for f in set_n:
        set_files.append(f)
    return set_files

In [7]:
files = image_set(data_dir, 'labels_round3')

In [8]:
files

['labels_round3_A001_2021-02-16-06-26-53.csv',
 'labels_round3_A002_2020-12-29-07-23-48.csv',
 'labels_round3_A004_2021-02-16-02-03-15.csv',
 'labels_round3_A005_2021-02-18-05-40-21.csv',
 'labels_round3_A006_2021-01-28-11-17-26.csv']

In [9]:
#df_set = [pd.read_csv(data_dir + filename, names=col_names, header = None) for filename in set1_files]
def extract_labels(directory, files):
    df_set = [pd.read_csv(directory + filename, names=col_names, header = None) for filename in files]
    df_reduce = reduce(lambda left, right: pd.merge(left, right, on='imname'), df_set)
    return df_reduce

In [10]:
label_data = extract_labels(data_dir, files)
label_data

,imname,annotator_x,annotator_y,annotator_x,annotator_y,annotator
0,18.jpg,[Non-appealing],[Appealing],[Non-appealing],[Non-appealing],[Appealing]
1,19.jpg,[Appealing],[Appealing],[Appealing],[Appealing],[Appealing]
2,20.jpg,[Non-appealing],[Appealing],[Non-appealing],[Non-appealing],[Appealing]
3,21.jpg,[Appealing],[Non-appealing],[Non-appealing],[Non-appealing],[Non-appealing]
4,22.jpg,[Appealing],[Appealing],[Appealing],[Appealing],[Appealing]
...,...,...,...,...,...,...
1005,985.jpg,[Appealing],[Appealing],[Non-appealing],[Non-appealing],[Appealing]
1006,986.jpg,[Non-appealing],[Appealing],[Non-appealing],[Non-appealing],[Appealing]
1007,987.jpg,[Non-appealing],[Non-appealing],[Non-appealing],[Non-appealing],[Appealing]
1008,988.jpg,[Appealing],[Non-appealing],[Non-appealing],[Non-appealing],[Non-appealing]


In [11]:
print(label_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1010 entries, 0 to 1009
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   imname       1010 non-null   object
 1   annotator_x  1010 non-null   object
 2   annotator_y  1010 non-null   object
 3   annotator_x  1010 non-null   object
 4   annotator_y  1010 non-null   object
 5   annotator    1010 non-null   object
dtypes: object(6)
memory usage: 55.2+ KB
None


In [12]:
df1 = label_data.apply(lambda row: sum(row[:]=='[Non-appealing]'), axis = 1)

In [13]:
df1 = pd.DataFrame(df1)

In [14]:
df1.columns = ['count_non-appealing']

In [15]:
df1

,count_non-appealing
0,3
1,0
2,3
3,4
4,0
...,...
1005,2
1006,3
1007,4
1008,4


In [16]:
df1 = label_data.join(df1, how = 'outer')

In [17]:
df1

,imname,annotator_x,annotator_y,annotator_x,annotator_y,annotator,count_non-appealing
0,18.jpg,[Non-appealing],[Appealing],[Appealing],[Non-appealing],[Non-appealing],3
1,19.jpg,[Appealing],[Appealing],[Appealing],[Appealing],[Appealing],0
2,20.jpg,[Non-appealing],[Appealing],[Appealing],[Non-appealing],[Non-appealing],3
3,21.jpg,[Appealing],[Non-appealing],[Non-appealing],[Non-appealing],[Non-appealing],4
4,22.jpg,[Appealing],[Appealing],[Appealing],[Appealing],[Appealing],0
...,...,...,...,...,...,...,...
1005,985.jpg,[Appealing],[Appealing],[Appealing],[Non-appealing],[Non-appealing],2
1006,986.jpg,[Non-appealing],[Appealing],[Appealing],[Non-appealing],[Non-appealing],3
1007,987.jpg,[Non-appealing],[Non-appealing],[Appealing],[Non-appealing],[Non-appealing],4
1008,988.jpg,[Appealing],[Non-appealing],[Non-appealing],[Non-appealing],[Non-appealing],4


In [18]:
df_new1 = (df1['count_non-appealing']/len(files))*100.0
df_new1 = pd.DataFrame(df_new1)
df_new1.columns = ['% Non-appealing']

In [19]:
df_new1

,% Non-appealing
0,60.0
1,0.0
2,60.0
3,80.0
4,0.0
...,...
1005,40.0
1006,60.0
1007,80.0
1008,80.0


In [20]:
df1 = df1.join(df_new1, how= 'outer')

In [22]:
df1

,imname,annotator_x,annotator_y,annotator_x,annotator_y,annotator,count_non-appealing,% Non-appealing
0,18.jpg,[Non-appealing],[Appealing],[Appealing],[Non-appealing],[Non-appealing],3,60.0
1,19.jpg,[Appealing],[Appealing],[Appealing],[Appealing],[Appealing],0,0.0
2,20.jpg,[Non-appealing],[Appealing],[Appealing],[Non-appealing],[Non-appealing],3,60.0
3,21.jpg,[Appealing],[Non-appealing],[Non-appealing],[Non-appealing],[Non-appealing],4,80.0
4,22.jpg,[Appealing],[Appealing],[Appealing],[Appealing],[Appealing],0,0.0
...,...,...,...,...,...,...,...,...
1005,985.jpg,[Appealing],[Appealing],[Appealing],[Non-appealing],[Non-appealing],2,40.0
1006,986.jpg,[Non-appealing],[Appealing],[Appealing],[Non-appealing],[Non-appealing],3,60.0
1007,987.jpg,[Non-appealing],[Non-appealing],[Appealing],[Non-appealing],[Non-appealing],4,80.0
1008,988.jpg,[Appealing],[Non-appealing],[Non-appealing],[Non-appealing],[Non-appealing],4,80.0


In [23]:
df2 = df1.apply(lambda row: sum(row[:]=='[Appealing]'), axis = 1)

In [24]:
df2 = pd.DataFrame(df2)

In [25]:
df2.columns = ['count_appealing']

In [26]:
df2

,count_appealing
0,2
1,5
2,2
3,1
4,5
...,...
1005,3
1006,2
1007,1
1008,1


In [27]:
df2 = df1.join(df2, how='outer')

In [28]:
df2

,imname,annotator_x,annotator_y,annotator_x,annotator_y,annotator,count_non-appealing,% Non-appealing,count_appealing
0,18.jpg,[Non-appealing],[Appealing],[Appealing],[Non-appealing],[Non-appealing],3,60.0,2
1,19.jpg,[Appealing],[Appealing],[Appealing],[Appealing],[Appealing],0,0.0,5
2,20.jpg,[Non-appealing],[Appealing],[Appealing],[Non-appealing],[Non-appealing],3,60.0,2
3,21.jpg,[Appealing],[Non-appealing],[Non-appealing],[Non-appealing],[Non-appealing],4,80.0,1
4,22.jpg,[Appealing],[Appealing],[Appealing],[Appealing],[Appealing],0,0.0,5
...,...,...,...,...,...,...,...,...,...
1005,985.jpg,[Appealing],[Appealing],[Appealing],[Non-appealing],[Non-appealing],2,40.0,3
1006,986.jpg,[Non-appealing],[Appealing],[Appealing],[Non-appealing],[Non-appealing],3,60.0,2
1007,987.jpg,[Non-appealing],[Non-appealing],[Appealing],[Non-appealing],[Non-appealing],4,80.0,1
1008,988.jpg,[Appealing],[Non-appealing],[Non-appealing],[Non-appealing],[Non-appealing],4,80.0,1


In [29]:
df2_newcol = (df2['count_appealing']/len(files))*100
df2_newcol = pd.DataFrame(df2_newcol)
df2_newcol.columns = ['% Appealing']

In [30]:
df2_newcol

,% Appealing
0,40.0
1,100.0
2,40.0
3,20.0
4,100.0
...,...
1005,60.0
1006,40.0
1007,20.0
1008,20.0


In [31]:
df2 = df2.join(df2_newcol, how='outer')

In [32]:
df2

,imname,annotator_x,annotator_y,annotator_x,annotator_y,annotator,count_non-appealing,% Non-appealing,count_appealing,% Appealing
0,18.jpg,[Non-appealing],[Appealing],[Appealing],[Non-appealing],[Non-appealing],3,60.0,2,40.0
1,19.jpg,[Appealing],[Appealing],[Appealing],[Appealing],[Appealing],0,0.0,5,100.0
2,20.jpg,[Non-appealing],[Appealing],[Appealing],[Non-appealing],[Non-appealing],3,60.0,2,40.0
3,21.jpg,[Appealing],[Non-appealing],[Non-appealing],[Non-appealing],[Non-appealing],4,80.0,1,20.0
4,22.jpg,[Appealing],[Appealing],[Appealing],[Appealing],[Appealing],0,0.0,5,100.0
...,...,...,...,...,...,...,...,...,...,...
1005,985.jpg,[Appealing],[Appealing],[Appealing],[Non-appealing],[Non-appealing],2,40.0,3,60.0
1006,986.jpg,[Non-appealing],[Appealing],[Appealing],[Non-appealing],[Non-appealing],3,60.0,2,40.0
1007,987.jpg,[Non-appealing],[Non-appealing],[Appealing],[Non-appealing],[Non-appealing],4,80.0,1,20.0
1008,988.jpg,[Appealing],[Non-appealing],[Non-appealing],[Non-appealing],[Non-appealing],4,80.0,1,20.0


In [33]:
df2.to_csv('labels_round3_team_annotations_analysis.csv', index=None)